In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import os
import torch
import numpy as np

from model.deeplatent import *
from model.networks import *
from utils.utils import *

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='3'

def modelnet_split(X,y):
    return X[:12137,:],X[12137:,:],y[:12137],y[12137:]

def smote(X,y):
    #------------SMOTE----------------
    #synthesize new data for those scarse categs
    from imblearn.over_sampling import SMOTE
    smt = SMOTE(random_state=42)
    X, y = smt.fit_resample(X, y)
    return X,y
    #-------------END-----------------
    
def filter_cate(X,y,topk = 7):
    cate_dict = {}
    for i in y:
        if i in cate_dict:
            cate_dict[i] += 1
        else:
            cate_dict[i] = 1
    cate_rank = sorted(cate_dict.items(),key=lambda x:x[1])[::-1][:topk]
    cate_allow_lst = [i[0] for i in cate_rank]
    
    print(cate_rank)
    print(cate_allow_lst)

    filtered_X = []
    filtered_y = []
    for i in range(X.shape[0]):
        if y[i] in cate_allow_lst:
            filtered_X.append(X[i,:])
            filtered_y.append(y[i])
    print(len(filtered_X))
    return np.array(filtered_X),filtered_y

def plot_confusion_matrix(y_true, y_pred, classes = range(40),
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    import matplotlib.pyplot as plt
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

In [ ]:

def randomForrest(X_train,X_test,y_train,y_test):
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=42)
    clf.fit(X_train, y_train) 
    y_pred =clf.predict(X_test)
    accuracy_score(y_test, y_pred)
    print("Training set score: %f" % clf.score(X_train, y_train))
    print("Test set score: %f" % clf.score(X_test, y_test))
    return y_pred
        
def mlp(X_train,X_test,y_train,y_test,iteration=3000):
    from sklearn.neural_network import MLPClassifier
    clf = MLPClassifier(max_iter=iteration, alpha=0.00007, hidden_layer_sizes = 1000,
                    solver='sgd', verbose=10,  random_state=42,tol=0.000000001)

    clf.fit(X_train, y_train)
    y_pred =clf.predict(X_test)
    accuracy_score(y_test, y_pred)
    print("Training set score: %f" % clf.score(X_train, y_train))
    print("Test set score: %f" % clf.score(X_test, y_test))
    print(classification_report(y_test,y_pred))
    return y_pred

def svm_linear(X_train,X_test,y_train,y_test,C=0.01):
    from sklearn.svm import LinearSVC
    clf = LinearSVC(random_state=0, tol=1e-5,C=C)
    clf.fit(X_train, y_train)
    y_pred =clf.predict(X_test)
    accuracy_score(y_test, y_pred)
    print("Training set score: %f" % clf.score(X_train, y_train))
    print("Test set score: %f" % clf.score(X_test, y_test))
    print(classification_report(y_test,y_pred))
    return y_pred

In [ ]:
####### test on modelnet sigma008 #######
from data.dataloader import ModelNet40_aligned
import glob
import os
import torch
root='/home/mmvc/mmvc-ny-nas/Yi_Shi/data/modelnets/ModelNet40_poisson'

ds = ModelNet40_aligned(root, None, mode='both')
y = ds.label.ravel()

latents = []
subset_dirs = ['results/md40_aligned_full/md40_subset%d'%i for i in range(13)]

for parent_dir in subset_dirs:
    save_name_test = os.path.join(parent_dir,'model_best_test_latents.pt')
    z_np = torch.load(save_name_test,map_location='cpu').detach().numpy()
    latents.append(z_np)
    print(z_np.shape)

X = np.concatenate(latents,axis=0)
train_num = ds.train_num
print(X.shape,y.shape,train_num)

X_train = X[:train_num,:]
X_test  = X[train_num:,:]
y_train = y[:train_num]
y_test  = y[train_num:]

#X_train, X_test, y_train, y_test = train_test_split(X, y)
#y_pred = svm_linear(X_train, X_test, y_train, y_test, C=0.01)
y_pred = mlp(X_train, X_test, y_train, y_test, iteration = 1000)